# Scikit learn



### Przeszukiwanie przestrzeni parametrów

Metody, z których korzystamy, można w jakimś stopniu poprawić, szukając optymalnej kombinacji parametrów.

Przykładem implementacji takiego przeszukiwania jest GridSearchCV.
http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

Poniżej pełny przykład wykorzystania (UWAGA! może być bardzo czasochłonny obliczeniowo)

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.feature_selection import SelectPercentile
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv('Tweets.csv')
df = df[df['airline_sentiment'] != 'neutral']

len(df)
X = df['text']
y = df['airline_sentiment']

X_train = X[:7000]
X_validate = X[7000:9000]
X_test = X[9000:]


y_train = y[:7000]
y_validate = y[7000:9000]
y_test = y[9000:]

pipeline = Pipeline([
       ('vect', TfidfVectorizer(stop_words='english')),
       ('sel', SelectPercentile()),
       ('clf', LinearSVC())
])

parameters = {
   'vect__max_df': (0.25, 0.5),
   'vect__ngram_range': ((1, 1), (1, 2), (1,3)),
   'vect__use_idf': (True, False),
   'sel__percentile': (10,30,50,100),
   'clf__C': (0.01, 1, 10),
   'clf__class_weight': ('balanced',None),
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=2, verbose=1, 
                           scoring='accuracy')


grid_search.fit(X_validate, y_validate)
print('Wyniki najlepszego modelu: %0.3f' % grid_search.best_score_)

best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print(param_name, best_parameters[param_name])
    
pipeline.set_params(**best_parameters)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
print(metrics.classification_report(y_test, y_pred, digits=3))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=2)]: Done  88 tasks      | elapsed:    8.1s
[Parallel(n_jobs=2)]: Done 388 tasks      | elapsed:   34.4s
[Parallel(n_jobs=2)]: Done 864 out of 864 | elapsed:  1.4min finished


Wyniki najlepszego modelu: 0.941
clf__C 1
clf__class_weight balanced
sel__percentile 30
vect__max_df 0.25
vect__ngram_range (1, 1)
vect__use_idf True
             precision    recall  f1-score   support

   negative      0.957     0.960     0.958      2163
   positive      0.766     0.754     0.760       378

avg / total      0.929     0.929     0.929      2541



Można też przetestować konkretną kombinację parametrów podając ją jako argument do pipeline. Robimy to tak:

In [ ]:
pipeline = Pipeline([
    ('cleantxt', TextCleaner()),
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    # ('clf', DecisionTreeClassifier()),#Score:0.12345
    ('clf', ExtraTreesClassifier(n_jobs=-1, n_estimators=50)),  # Score:0.2345
    # ('dense', DenseTransformer()),
    # ('clf', GaussianNB())#Score:0.62
])

pipe_params = {
    'clf__class_weight': 'balanced'
    , 'clf__random_state': 42
    , 'tfidf__norm': 'l2'
    , 'tfidf__use_idf': True
    , 'vect__max_df': .7
    , 'vect__max_features': 10000
    , 'vect__ngram_range': (1, 2)
    , 'vect__strip_accents': 'unicode'
    , 'vect__stop_words': stops
}

pipeline.set_params(**pipe_params)

# przepisac parametry pipe_params z wynikow GridSearchCV

pipeline.fit(X_train, Y_train)  # dzialamy na 70% danych zeby wytrenowac ostateczny model

Y_predicted = pipeline.predict(X_test) # przewidujemy etykiety na 10% zbiorze testowym

# TODO policzyc jakosc tego modelu (patrz scikit_1)

# nieliniowy SVM:
# http://scikit-learn.org/stable/modules/generated/sklearn.svm.NuSVC.html
import sklearn.svm.NuSVC
clf = NuSVC(nu=0.5, kernel='rbf', class_weight='balanced')

Zadania

1) oprócz train i test, wprowadź jeszcze validate - (jako train - 70%, validate - 20%, test - 10%). Zmodyfikuj powyższy kod, aby dane do  GridSearchCV pochodziły z podziału X_validate i Y_validate.
Uruchom grid search i najlepszą kombinację parametrów przetestuj na Y_test, trenując tym razem model na 80% zbiorze X_train.

2) Wypróbuj klasyfikatory: RandomForest, LogisticRegression, SVM

3) Spróbuj dodać do pipeline normalizację danych, przetestuj wyniki

4) Oceń wybrany klasyfikator z najlepszą kombinacją parametrów używając metody 10-krotnej walidacji krzyżowej. Policz średnią precyzję, recall i F1 dla foldów. Jak różnią się otrzymane wyniki od uzyskanych poprzednio?
http://scikit-learn.org/stable/modules/cross_validation.html